In [1]:
import numpy as np
import pandas as pd
import os
import sys
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.image as mpimg
import optuna
import random
import joblib
import math
import ast
import logging
from msig import Motif, NullModel
from config import RESULTS_MOTIF_DIR, RESULTS_DIR, IMAGES_DIR, DATA_DIR, DATASET_PATH, TOWNSHIP_NAME, VARIABLES, NORMALIZE_FLAGS, STUMPY_EXCL_ZONE_DENOM, TOP_K_MP, INCLUDE, NORMALIZE, SUBSQUENCES_LENGTHS, NTOP_MOTIFS, MOTIF_SIZE

optuna.logging.set_verbosity(optuna.logging.WARNING)

print(f"Results will be saved in: {RESULTS_DIR}")
print(f"Images will be saved in: {IMAGES_DIR}")
print(f"Data will be accessed from: {DATA_DIR}")


if '__file__' in globals():
    # For standalone scripts
    base_dir = os.path.dirname(__file__)
else:
    # For Jupyter or interactive environments
    base_dir = os.getcwd()

# Add the parent directory of `utils` to the Python path
sys.path.append(os.path.abspath(os.path.join(base_dir, "../")))

2025-02-04 17:12:14,886 - INFO - Results will be saved in: /home/mgsilva/motifpred/results/populationdensity
2025-02-04 17:12:14,887 - INFO - Images will be saved in: /home/mgsilva/motifpred/images/populationdensity
2025-02-04 17:12:14,887 - INFO - Data will be accessed from: /home/mgsilva/motifpred/data/populationdensity


Results will be saved in: /home/mgsilva/motifpred/results/populationdensity
Images will be saved in: /home/mgsilva/motifpred/images/populationdensity
Data will be accessed from: /home/mgsilva/motifpred/data/populationdensity


In [2]:
mp_stats_table = pd.read_csv(
    RESULTS_DIR / f"mp_stats_table_normalized_{NORMALIZE}_top_{TOP_K_MP}.csv"
)
mp_stats_table = mp_stats_table[mp_stats_table["m"] == MOTIF_SIZE]
top_motifs = mp_stats_table.sort_values(by=["#Matches", "ID"], ascending=[False, True]).head(NTOP_MOTIFS)
top_motifs = top_motifs[["m", "Indices"]]

In [3]:
# read csv
data_df = pd.read_csv(
    DATASET_PATH,
    parse_dates=["one_time"],
    date_format="%Y-%m-%d %H:%M:%S",
    index_col=0,
)

data_df = data_df[data_df["township_name"] == TOWNSHIP_NAME]
#set index to one_time and township_name
data_df = data_df.set_index(["one_time", "township_name"]).sort_index()[VARIABLES]
data = data_df.to_numpy().T
data_univar = data[0]
data_df

,,sum_terminals
one_time,township_name,
2021-09-15 00:00:00,Avenidas Novas,260700.0
2021-09-15 01:00:00,Avenidas Novas,276675.0
2021-09-15 02:00:00,Avenidas Novas,284563.0
2021-09-15 03:00:00,Avenidas Novas,279563.0
2021-09-15 04:00:00,Avenidas Novas,281460.0
...,...,...
2021-11-30 19:00:00,Avenidas Novas,391367.0
2021-11-30 20:00:00,Avenidas Novas,352361.0
2021-11-30 21:00:00,Avenidas Novas,388246.0


In [4]:
# Import shared setup
from utils.setup import seed, device, early_stopper, pipeline, test_tensor

# Example usage
print(f"Device: {device}")
test_tensor()

Device: cuda
tensor([[0.3126, 0.3791, 0.3087],
        [0.0736, 0.4216, 0.0691],
        [0.2332, 0.4047, 0.2162],
        [0.9927, 0.4128, 0.5938],
        [0.6128, 0.1519, 0.0453]])


In [5]:
from utils.utils import create_dataset
from utils.train_pipeline import run_optuna_study
from utils.utils import get_best_model_results_traindevtest, plot_best_model_results_traindevtest
from models.ffnn_pytorch import FFNN
from utils.utils import plot_preds_vs_truevalues
from utils.train_pipeline import get_preds_best_config_train_val_test

lookback_period = 24*7*3 #window size
step = 1 #step size for the sliding window
forecast_period = 24*2 #forward window size
test_losses_list = []
test_mae_list = []
test_rmse_list = []

# Loop through each of the top 10 motifs
for i, top_motif in top_motifs.iterrows():

    motif_indexes = sorted(ast.literal_eval(top_motif["Indices"]))
    
    print(f"Evaluating motif {i+1} with size {MOTIF_SIZE} and {len(motif_indexes)} indexes")
    
    # Create dataset for the current motif
    X_series, X_indices, X_mask, y = create_dataset(data, lookback_period, step, forecast_period, motif_indexes, MOTIF_SIZE)

    # X_series, X2, and y are now PyTorch tensors
    print("X_series shape:", X_series.shape)  # Expected shape: (num_samples, lookback_period, num_features)
    print("X_indices shape:", X_indices.shape)  # Expected shape: (num_samples, max_motif_length_in_window, 1)
    print("X_mask shape:", X_mask.shape)  # Expected shape: (num_samples, max_motif_length_in_window)
    print("y shape:", y.shape)    # Expected shape: (num_samples, 1)
    
    # Define the model and run the Optuna study
    n_trials = 100
    num_epochs = 500
    model_type = "FFNN"
    model_name = "FFNNSeries"

    suggestion_dict = {
        "learning_rate": {
            "type": "float",
            "args": [1e-5, 1e-3], 
            "kwargs": {"log": True} 
        },
        "num_layers": {
            "type": "categorical",
            "args": [[1, 2, 3, 4]] 
        },        
        "batch_size": {
            "type": "categorical",
            "args": [[4, 8, 16, 32]]
        }
    }

    model_params_keys = ["hidden_sizes_list"]
    model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{i+1}")
    os.makedirs(model_results_dir, exist_ok=True)  

    X = {"X_series": X_series}
    run_optuna_study(pipeline.run_train_val_test, eval(model_type), model_type, suggestion_dict, model_params_keys, seed, X, y, NORMALIZE_FLAGS, model_results_dir, n_trials=n_trials, num_epochs=num_epochs)

    study = joblib.load(os.path.join(model_results_dir, "study.pkl"))
    train_losses, val_losses, best_epoch, test_loss, test_mae, test_rmse = get_best_model_results_traindevtest(study)

    print(f"Best epoch: {best_epoch}")
    print(f"Test Loss: {test_loss}, Test MAE: {test_mae}, Test RMSE: {test_rmse}")

    test_losses_list.append(test_loss)
    test_mae_list.append(test_mae)
    test_rmse_list.append(test_rmse)
    
    epochs_train_losses, epochs_val_losses, val_losses, test_losses, test_mae, test_rmse, all_predictions, all_true_values = get_preds_best_config_train_val_test(study, pipeline, eval(model_type), model_type, model_params_keys, num_epochs=num_epochs, seed=seed, X=X, y=y, normalize_flags=NORMALIZE_FLAGS)
    #plot_best_model_results_traindevtest( study.trials_dataframe(),
    #    save_path=os.path.join(IMAGES_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{i+1}_best_results.png")
    #)    
    #plot_preds_vs_truevalues(np.ravel(all_true_values), np.ravel(all_predictions), fold=0, save_path=os.path.join(IMAGES_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{i+1}_fold_{0}_predictions.png"))


# Convert lists to numpy arrays for easier calculations
test_losses_array = np.array(test_losses_list)
test_mae_array = np.array(test_mae_list)
test_rmse_array = np.array(test_rmse_list)

# Calculate mean and standard deviation
mean_test_loss = np.mean(test_losses_array)
std_test_loss = np.std(test_losses_array)

mean_test_mae = np.mean(test_mae_array)
std_test_mae = np.std(test_mae_array)

mean_test_rmse = np.mean(test_rmse_array)
std_test_rmse = np.std(test_rmse_array)

# Print aggregated results
print(f"Aggregated Results Across Top 5 Motifs:")
print(f"Mean Test Loss: {mean_test_loss} ± {std_test_loss}")
print(f"Mean Test MAE: {mean_test_mae} ± {std_test_mae}")
print(f"Mean Test RMSE: {mean_test_rmse} ± {std_test_rmse}")


Evaluating motif 27 with size 12 and 58 indexes
X_series shape: torch.Size([1173, 504, 1])
X_indices shape: torch.Size([1173, 18, 1])
X_mask shape: torch.Size([1173, 504])
y shape: torch.Size([1173, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 359.17095947265625, Test MAE: 15.468602180480957, Test RMSE: 18.951807022094727
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 6 with size 12 and 47 indexes
X_series shape: torch.Size([1032, 504, 1])
X_indices shape: torch.Size([1032, 16, 1])
X_mask shape: torch.Size([1032, 504])
y shape: torch.Size([1032, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 378.9913330078125, Test MAE: 15.857345581054688, Test RMSE: 19.46769905090332
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 24 with size 12 and 46 indexes
X_series shape: torch.Size([1122, 504, 1])
X_indices shape: torch.Size([1122, 16, 1])
X_mask shape: torch.Size([1122, 504])
y shape: torch.Size([1122, 1])


...

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 477.03369140625, Test MAE: 18.516128540039062, Test RMSE: 21.841100692749023
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 12 with size 12 and 44 indexes
X_series shape: torch.Size([1050, 504, 1])
X_indices shape: torch.Size([1050, 15, 1])
X_mask shape: torch.Size([1050, 504])
y shape: torch.Size([1050, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 531.3179321289062, Test MAE: 19.097576141357422, Test RMSE: 23.050334930419922
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 7 with size 12 and 38 indexes
X_series shape: torch.Size([979, 504, 1])
X_indices shape: torch.Size([979, 12, 1])
X_mask shape: torch.Size([979, 504])
y shape: torch.Size([979, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 583.5614013671875, Test MAE: 20.03963851928711, Test RMSE: 24.157014846801758
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Aggregated Results Across Top 5 Motifs:
Mean Test Loss: 466.0150634765625 ± 86.24593740260501
Mean Test MAE: 17.79585819244385 ± 1.8122781705361783
Mean Test RMSE: 21.49359130859375 ± 2.010121649001188


In [6]:
from utils.utils import create_dataset
from utils.train_pipeline import run_optuna_study
from utils.utils import get_best_model_results_traindevtest, plot_best_model_results_traindevtest
from models.ffnn_pytorch import FFNN
from utils.utils import plot_preds_vs_truevalues
from utils.train_pipeline import get_preds_best_config_train_val_test

lookback_period = 24*7*3 #window size
step = 1 #step size for the sliding window
forecast_period = 24*2 #forward window size
test_losses_list = []
test_mae_list = []
test_rmse_list = []

# Loop through each of the top 10 motifs
for i, top_motif in top_motifs.iterrows():

    motif_indexes = sorted(ast.literal_eval(top_motif["Indices"]))
    
    print(f"Evaluating motif {i+1} with size {MOTIF_SIZE} and {len(motif_indexes)} indexes")
    
    # Create dataset for the current motif
    X_series, X_indices, X_mask, y = create_dataset(data, lookback_period, step, forecast_period, motif_indexes, MOTIF_SIZE)

    # X_series, X2, and y are now PyTorch tensors
    print("X_series shape:", X_series.shape)  # Expected shape: (num_samples, lookback_period, num_features)
    print("X_indices shape:", X_indices.shape)  # Expected shape: (num_samples, max_motif_length_in_window, 1)
    print("X_mask shape:", X_mask.shape)  # Expected shape: (num_samples, max_motif_length_in_window)
    print("y shape:", y.shape)    # Expected shape: (num_samples, 1)
    
    # Define the model and run the Optuna study
    n_trials = 100
    num_epochs = 500
    model_type = "FFNN"
    model_name = "FFNNSeries_Masking"

    suggestion_dict = {
        "learning_rate": {
            "type": "float",
            "args": [1e-5, 1e-3], 
            "kwargs": {"log": True} 
        },
        "num_layers": {
            "type": "categorical",
            "args": [[1, 2, 3, 4]] 
        },        
        "batch_size": {
            "type": "categorical",
            "args": [[4, 8, 16, 32]]
        }
    }

    model_params_keys = ["hidden_sizes_list"]
    model_results_dir = os.path.join(RESULTS_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{i+1}")
    os.makedirs(model_results_dir, exist_ok=True)  

    X = {"X_series": X_series, "X_mask": X_mask}
    run_optuna_study(pipeline.run_train_val_test, eval(model_type), model_type, suggestion_dict, model_params_keys, seed, X, y, NORMALIZE_FLAGS, model_results_dir, n_trials=n_trials, num_epochs=num_epochs)

    study = joblib.load(os.path.join(model_results_dir, "study.pkl"))
    train_losses, val_losses, best_epoch, test_loss, test_mae, test_rmse = get_best_model_results_traindevtest(study)

    print(f"Best epoch: {best_epoch}")
    print(f"Test Loss: {test_loss}, Test MAE: {test_mae}, Test RMSE: {test_rmse}")

    test_losses_list.append(test_loss)
    test_mae_list.append(test_mae)
    test_rmse_list.append(test_rmse)
    
    epochs_train_losses, epochs_val_losses, val_losses, test_losses, test_mae, test_rmse, all_predictions, all_true_values = get_preds_best_config_train_val_test(study, pipeline, eval(model_type), model_type, model_params_keys, num_epochs=num_epochs, seed=seed, X=X, y=y, normalize_flags=NORMALIZE_FLAGS)
    #plot_best_model_results_traindevtest( study.trials_dataframe(),
    #    save_path=os.path.join(IMAGES_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{i+1}_best_results.png")
    #)    
    #plot_preds_vs_truevalues(np.ravel(all_true_values), np.ravel(all_predictions), fold=0, save_path=os.path.join(IMAGES_DIR, f"{model_name}_{n_trials}_trials_{num_epochs}_epochs_motif_{i+1}_fold_{0}_predictions.png"))


# Convert lists to numpy arrays for easier calculations
test_losses_array = np.array(test_losses_list)
test_mae_array = np.array(test_mae_list)
test_rmse_array = np.array(test_rmse_list)

# Calculate mean and standard deviation
mean_test_loss = np.mean(test_losses_array)
std_test_loss = np.std(test_losses_array)

mean_test_mae = np.mean(test_mae_array)
std_test_mae = np.std(test_mae_array)

mean_test_rmse = np.mean(test_rmse_array)
std_test_rmse = np.std(test_rmse_array)

# Print aggregated results
print(f"Aggregated Results Across Top 5 Motifs:")
print(f"Mean Test Loss: {mean_test_loss} ± {std_test_loss}")
print(f"Mean Test MAE: {mean_test_mae} ± {std_test_mae}")
print(f"Mean Test RMSE: {mean_test_rmse} ± {std_test_rmse}")


Evaluating motif 27 with size 12 and 58 indexes
X_series shape: torch.Size([1173, 504, 1])
X_indices shape: torch.Size([1173, 18, 1])
X_mask shape: torch.Size([1173, 504])
y shape: torch.Size([1173, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 358.5529479980469, Test MAE: 15.445839881896973, Test RMSE: 18.93549346923828
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 6 with size 12 and 47 indexes
X_series shape: torch.Size([1032, 504, 1])
X_indices shape: torch.Size([1032, 16, 1])
X_mask shape: torch.Size([1032, 504])
y shape: torch.Size([1032, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 378.52447509765625, Test MAE: 15.843049049377441, Test RMSE: 19.455705642700195
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 24 with size 12 and 46 indexes
X_series shape: torch.Size([1122, 504, 1])
X_indices shape: torch.Size([1122, 16, 1])
X_mask shape: torch.Size([1122, 504])
y shape: torch.Size([1122, 1])


...

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 476.3528137207031, Test MAE: 18.49933433532715, Test RMSE: 21.82550811767578
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 12 with size 12 and 44 indexes
X_series shape: torch.Size([1050, 504, 1])
X_indices shape: torch.Size([1050, 15, 1])
X_mask shape: torch.Size([1050, 504])
y shape: torch.Size([1050, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 530.8046264648438, Test MAE: 19.084659576416016, Test RMSE: 23.03919792175293
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Evaluating motif 7 with size 12 and 38 indexes
X_series shape: torch.Size([979, 504, 1])
X_indices shape: torch.Size([979, 12, 1])
X_mask shape: torch.Size([979, 504])
y shape: torch.Size([979, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

Training completed all epochs. Best epoch was 0
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Best epoch: 0
Test Loss: 583.0713500976562, Test MAE: 20.02541732788086, Test RMSE: 24.146869659423828
Best hyperparameters: {'learning_rate': 2.6565450821928437e-05, 'num_layers': 3, 'batch_size': 32, 'hidden_size_layer_0': 64, 'hidden_size_layer_1': 64, 'hidden_size_layer_2': 16}
Training completed all epochs. Best epoch was 0
Aggregated Results Across Top 5 Motifs:
Mean Test Loss: 465.46124267578125 ± 86.26460052704178
Mean Test MAE: 17.779660034179688 ± 1.8144728896756086
Mean Test RMSE: 21.4805549621582 ± 2.0117130704503445
